In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions import Categorical

In [2]:

class Model_Drop(nn.Module):
    def __init__(self):

        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
        self.drop_layer = nn.Dropout(p=0.50)

    def last_hidden_layer_output(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.view(-1, 7*7*64)
        x = self.drop_layer((F.relu(self.fc1(x))))
        x = self.drop_layer((F.relu(self.fc2(x))))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_cnn = Model_Drop()
model_cnn = model_cnn.to(device)

In [4]:
batch_size = 128

mnist_train = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 123182182.80it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 117607469.73it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 55069844.86it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 1987120.97it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
criterion = nn.CrossEntropyLoss()

learning_rate = 0.002

optimizer = optim.Adam(model_cnn.parameters(), lr=learning_rate)

In [6]:

def train(model, train_dataloader):

    model = model.to(device)
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0

    for data, target in train_dataloader:

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()

    train_loss = train_running_loss / len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct / len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')

    return train_loss, train_accuracy

In [7]:
# validation function
def test(model, test_dataloader):
    model = model.to(device)
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0

    for data, target in test_dataloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)

        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()

    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)

    return val_loss, val_accuracy

In [8]:
print("BEGIN TRAINING")

train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []

for epoch in range(15):

    train_epoch_loss, train_epoch_accuracy = train(model_cnn, train_loader)
    val_epoch_loss, val_epoch_accuracy = test(model_cnn, test_loader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)


torch.save(model_cnn.state_dict(), "model_cnn_mnist_digit.pt")

print("done")

BEGIN TRAINING
Train Loss: 0.0022, Train Acc: 91.20
Train Loss: 0.0007, Train Acc: 97.54
Train Loss: 0.0005, Train Acc: 98.19
Train Loss: 0.0005, Train Acc: 98.39
Train Loss: 0.0004, Train Acc: 98.67
Train Loss: 0.0003, Train Acc: 98.86
Train Loss: 0.0003, Train Acc: 98.95
Train Loss: 0.0003, Train Acc: 98.91
Train Loss: 0.0003, Train Acc: 99.09
Train Loss: 0.0002, Train Acc: 99.14
Train Loss: 0.0002, Train Acc: 99.30
Train Loss: 0.0002, Train Acc: 99.24
Train Loss: 0.0002, Train Acc: 99.28
Train Loss: 0.0002, Train Acc: 99.36
Train Loss: 0.0002, Train Acc: 99.28
done


In [9]:
softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

In [10]:

def uncertainty_quantification(image,model,T):

    image = image.detach()

    item_count = image.shape[0]

    dropout_predictions = torch.zeros([T,item_count,10])

    for i in range(T):

        enable_dropout(model)
        output = model((image))
        output = softmax(output)

        dropout_predictions[i] = output

    variance = torch.var(dropout_predictions, dim=0)

    var = variance.mean(1,True)
    var = var.reshape(1,item_count)

    return var.detach()

In [12]:
for X,y in test_loader:
    X,y = X.to(device), y.to(device)
    break

attacked_sample = 9

for i, (image,label) in enumerate(test_loader):
    image, label = image.to(device), label.to(device)

    if i == 0:

        sample_img = image[attacked_sample]
        sample_img = torch.unsqueeze(sample_img,0)
        sample_label = label[attacked_sample]
        sample_label = torch.unsqueeze(sample_label, 0)
        break

with torch.no_grad():
    o = model_cnn(sample_img)
    o = softmax(o)

prediction = o.data.max(1, keepdim=True)[1]
print("label is ", sample_label)
print("prediction is ", prediction )

print("quantified uncertainty is ", uncertainty_quantification(sample_img,model_cnn,50))

label is  tensor([9], device='cuda:0')
prediction is  tensor([[9]], device='cuda:0')
quantified uncertainty is  tensor([[2.0602e-08]])


In [13]:
!pip install foolbox


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00


In [14]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

eps = 0.18

In [15]:
attack = LinfDeepFoolAttack()
fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
raw_advs, clipped_advs, success = attack(fmodel, sample_img, sample_label, epsilons=[eps])
pert = torch.tensor(clipped_advs[0])

with torch.no_grad():
    o = model_cnn(pert)
    o = softmax(o)

prediction = o.data.max(1, keepdim=True)[1]
print("label is ", sample_label)
print("prediction is ", prediction )

print("quantified uncertainty is ", uncertainty_quantification(pert,model_cnn,50))

label is  tensor([9], device='cuda:0')
prediction is  tensor([[7]], device='cuda:0')
quantified uncertainty is  tensor([[0.0225]])


<ipython-input-15-97f99ae1a6a2>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert = torch.tensor(clipped_advs[0])


In [16]:
attack = LinfBasicIterativeAttack()
fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
raw_advs, clipped_advs, success = attack(fmodel, sample_img, sample_label, epsilons=[eps])
pert = torch.tensor(clipped_advs[0])

with torch.no_grad():
    o = model_cnn(pert)
    o = softmax(o)

prediction = o.data.max(1, keepdim=True)[1]
print("label is ", sample_label)
print("prediction is ", prediction )

print("quantified uncertainty is ", uncertainty_quantification(pert,model_cnn,50))

label is  tensor([9], device='cuda:0')
prediction is  tensor([[4]], device='cuda:0')
quantified uncertainty is  tensor([[3.8488e-07]])


<ipython-input-16-8677900afcf0>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert = torch.tensor(clipped_advs[0])


In [ ]:
def predict_uncertainties(model, image, T=50):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    image = image.to(device)

    image = image.detach()
    item_count = image.shape[0]

    dropout_predictions = torch.zeros([T, item_count, 10])

    for t in range(T):

        enable_dropout(model)

        with torch.no_grad():
            output = model(image)

        output_prob = F.softmax(output, dim=1) #shape is 1x10 if item_count is 1 (only one image in input batch)
        dropout_predictions[t] = output_prob

    mean = torch.mean(dropout_predictions, dim=0)

    entropy = Categorical(probs=mean).entropy()

    pred_mean = mean

    aleatoric = torch.zeros([item_count,10,10])
    epistemic = torch.zeros([item_count,10,10])

    for t in range(T):

        pred_t = dropout_predictions[t]

        aleatoric += torch.diag_embed(pred_t, offset=0, dim1=-2, dim2=-1) - pred_t[:, :, None] @ pred_t[:, None, :]
        epistemic += (pred_t - pred_mean)[:, :, None] @ (pred_t - pred_mean)[:, None, :]

    aleatoric = aleatoric / T #both of them are of shape item_count x 10x10
    epistemic = epistemic / T #both of them are of shape item_count x 10x10

    aleatoric = torch.diagonal(aleatoric, 0, dim1=-2, dim2=-1)
    epistemic = torch.diagonal(epistemic, 0, dim1=-2, dim2=-1)

    aleatoric = torch.mean(aleatoric,1,True)
    epistemic = torch.mean(epistemic, 1, True)

    model.eval()

    return aleatoric.transpose_(0, 1)[0], epistemic.transpose_(0, 1)[0], entropy